In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
books = pd.read_csv('data/book/Books.csv')
users = pd.read_csv('data/book/Users.csv')
ratings = pd.read_csv('data/book/Ratings.csv')

C:\Users\arbas\AppData\Local\Temp\ipykernel_19300\1614979886.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('data/book/Books.csv')


In [4]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [5]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [7]:
print(books.shape)
print(users.shape)
print(ratings.shape)

(271360, 8)
(278858, 3)
(1149780, 3)


In [8]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [9]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

**Observation:** To many missing ages, in users data.

#### EDA

### Creating Popularity Based Recommender System

We'll display top 20 books, with highest average rating, where each book has minimum of 250 votes.

In [37]:
ratings_with_books = ratings.merge(right=books, on='ISBN')

In [38]:
rating_df = ratings_with_books.groupby('Book-Title').count()['Book-Rating'].reset_index()
rating_df.rename(columns={'Book-Rating': 'Rating-Count'}, inplace=True)

In [39]:
avg_rating_df = pd.DataFrame(ratings_with_books.groupby('Book-Title')['Book-Rating'].mean()).reset_index()
avg_rating_df.rename(columns={'Book-Rating': 'Average-Rating'}, inplace=True)

In [40]:
avg_rating_df.head()

,Book-Title,Average-Rating
0,A Light in the Storm: The Civil War Diary of ...,2.25
1,Always Have Popsicles,0.00
2,Apple Magic (The Collector's series),0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00
4,Beyond IBM: Leadership Marketing and Finance ...,0.00


In [41]:
popularity_df = rating_df.merge(right=avg_rating_df, on='Book-Title')

In [42]:
popularity_df

,Book-Title,Rating-Count,Average-Rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [43]:
popularity_df = popularity_df[popularity_df['Rating-Count']>=250].sort_values(by='Average-Rating', ascending=False).head(50)

In [44]:
popularity_df.shape

(50, 3)

In [47]:
popularity_df = popularity_df.merge(right=books, on='Book-Title').drop_duplicates('Book-Title')[['Book-Title', 'Book-Author', 'Year-Of-Publication', 'Image-URL-M', 'Rating-Count', 'Average-Rating']]

### Creating Collaborative Filtering Based Recommender System

- Consider those users, who have rated atleast 200 books, i.e. experienced users.
- Consider those books that have minimum 50 no of ratings.

In [55]:
temp = ratings_with_books.groupby('User-ID').count()['Book-Title'] >= 200
temp[temp]

User-ID
254       True
2276      True
2766      True
2977      True
3363      True
          ... 
274308    True
275970    True
277427    True
277639    True
278418    True
Name: Book-Title, Length: 816, dtype: bool

**Observation:** Total only 816 users have rated >= 200 times.

In [57]:
user_indices = temp[temp].index

In [63]:
filtered_rating = ratings_with_books[ratings_with_books['User-ID'].isin(user_indices)]

In [71]:
temp3 = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
temp3[temp3]

Book-Title
1984                                                                 True
1st to Die: A Novel                                                  True
2nd Chance                                                           True
4 Blondes                                                            True
A Bend in the Road                                                   True
                                                                     ... 
Year of Wonders                                                      True
You Belong To Me                                                     True
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values    True
Zoya                                                                 True
\O\" Is for Outlaw"                                                  True
Name: Book-Rating, Length: 707, dtype: bool

In [72]:
popular_books = temp3[temp3].index

In [75]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(popular_books)]

In [78]:
final_ratings.duplicated().sum()

0

|       | user1 | user2 | user3 | user4 |
|-------|-------|-------|-------|-------|
| book1 |   1   |   5   |   1   |   2   |
| book2 |   2   |   5   |   3   |   4   |
| book3 |   2   |   5   |   5   |   3   |
| book4 |   1   |   5   |   1   |   2   |

- This matrix is tells us what `useri`, rated `bookj`.

In [81]:
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [82]:
pt.fillna(0,inplace=True)
# Try with KNN imputer maybe

In [88]:
# Finding Simmilarity between books
from sklearn.metrics.pairwise import cosine_similarity
simmilarity_scores = cosine_similarity(pt)

In [89]:
simmilarity_scores[0]

array([1.        , 0.0999137 , 0.01189468, 0.        , 0.05229234,
       0.02703559, 0.08005247, 0.133798  , 0.03177829, 0.03573298,
       0.0226271 , 0.06592522, 0.020304  , 0.09425025, 0.1304464 ,
       0.08089641, 0.10866788, 0.0496928 , 0.02453052, 0.11405415,
       0.        , 0.13965274, 0.07645776, 0.05992325, 0.08499677,
       0.        , 0.06829594, 0.13307519, 0.07404925, 0.11854319,
       0.007483  , 0.01435345, 0.        , 0.07761015, 0.04406341,
       0.0151626 , 0.09251799, 0.01776199, 0.02543351, 0.07778745,
       0.11379679, 0.0554492 , 0.08139372, 0.08254087, 0.08560054,
       0.05350252, 0.0534403 , 0.26331669, 0.09527704, 0.05861375,
       0.08728505, 0.06575169, 0.        , 0.04353225, 0.01871487,
       0.        , 0.05484345, 0.00531237, 0.07674541, 0.05085288,
       0.18422052, 0.        , 0.01208759, 0.0290784 , 0.04169477,
       0.12354122, 0.16140808, 0.        , 0.1301383 , 0.06445396,
       0.        , 0.        , 0.        , 0.10686088, 0.02734

This tells us how much book no 0 is simmilar to other books in the dataframe.

In [93]:
def recommend_books(book_name):
    book_index = np.where(pt.index==book_name)[0][0]
    distances = simmilarity_scores[book_index]
    suggestions = sorted(enumerate(distances), key=lambda x: x[1], reverse=True)[1:6] # 0th index is the book itself
    for i in suggestions:
        print(i[0], pt.index[i[0]])
    return suggestions

In [96]:
suggestions = recommend_books('1984')

47 Animal Farm
546 The Handmaid's Tale
82 Brave New World
635 The Vampire Lestat (Vampire Chronicles, Book II)
552 The Hours : A Novel
